In [2]:
import pandas as pd
import numpy as np

In [3]:
twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = twitter_archive_df.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

1: 892420643555336193
Fail
2: 892177421306343426
Fail
3: 891815181378084864


In [4]:
image_predictions_df = pd.read_csv('image_predictions.tsv', sep='\t')

In [5]:
json_data = pd.read_json('tweet-json.txt', lines=True)

In [6]:
final_json_data = json_data[['id', 'retweet_count', 'favorite_count']]

In [ ]:
image_predictions_df.info()

In [ ]:
final_json_data.info()

## Data Assessment - Quality Issues (8)

#### Twitter Archive Dataframe
- tweet_id column is integer datatype
- in_reply_to_status_id is float datatype 
- in_reply_to_user_id is float datatype
- timestamp is object datatype
- retweeted_status_id is float datatype
- retweeted_status_user_id is float datatype
- one of the denominators has a rating of "0"
- many of the dog names are "none" or they don't make sense


In [ ]:
twitter_archive_df.sample(2)

In [ ]:
twitter_archive_df['rating_denominator'].value_counts()

In [ ]:
twitter_archive_df.info()

In [ ]:
# twitter_archive_df.loc[twitter_archive_df['rating_denominator'] != 10]

In [ ]:
twitter_archive_df['name'].value_counts()

In [ ]:
twitter_archive_df[twitter_archive_df["name"].duplicated()]

In [ ]:
twitter_archive_df.sample(10)

## Data Assessment - Tidiness Issues (2)
- dog stage should be a single column, not 4
- all three tables can be joined into one


## Cleaning

#### Define


- change tweet_id column to object datatype
- change in_reply_to_status_id to object datatype 
- change in_reply_to_user_id to object datatype
- change timestamp to datetime datatype
- change retweeted_status_id to object datatype
- change retweeted_status_user_id to object datatype
- for the row where denominator has a rating of "0", change it "10"
- convert rows (where the dog name is 'None') to np.nan
- convert rows (where the dog name begins in lower case) to np.nan
- merge dog stage columns into one column
- merge all tables into one master table


#### Code

In [7]:
# 0. make a copy of the dataframe we want to clean, twitter_archive_df
twitter_archive_df_clean = twitter_archive_df.copy()

In [8]:
# 1. change id columns to object datatype
twitter_archive_df_clean = twitter_archive_df_clean.astype({"tweet_id": object,
                "in_reply_to_status_id": object,
                "in_reply_to_user_id": object,
                "retweeted_status_id": object,
                "retweeted_status_user_id": object})

In [9]:
# 2. change timestamp column to datetime datatype
twitter_archive_df_clean["timestamp"]= pd.to_datetime(twitter_archive_df_clean["timestamp"])

In [11]:
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       78 non-null     object             
 2   in_reply_to_user_id         78 non-null     object             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    object             
 7   retweeted_status_user_id    181 non-null    object             
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

In [10]:
# 3. for the row where denominator has a rating of "0", change it "10"
twitter_archive_df_clean.at[313,'rating_denominator']=10

In [11]:
# 4. convert rows (where the dog name begins in lower case) to np.nan
# first make a mask
mask = twitter_archive_df_clean.name.str.contains('^[a-z]', regex = True)

# then change lower case names to nan
twitter_archive_df_clean.loc[mask,'name'] = np.nan

In [12]:
# 5. convert rows (where the dog name is 'None') to np.nan
twitter_archive_df_clean.name.replace('None', np.nan, inplace=True)

In [ ]:
# 6. merge dog stage columns into one column

In [ ]:
# first, we change cells with 'None' to ''
# then, make the new column equal to old column1 + old column2.. etc
# finally drop the old columns, and deal with multivalue cells

In [13]:
twitter_archive_df_clean.doggo.replace('None', '', inplace=True)
twitter_archive_df_clean.floofer.replace('None', '', inplace=True)
twitter_archive_df_clean.pupper.replace('None', '', inplace=True)
twitter_archive_df_clean.puppo.replace('None', '', inplace=True)

In [14]:
twitter_archive_df_clean['stage'] = twitter_archive_df_clean.doggo + \
    twitter_archive_df_clean.floofer + \
    twitter_archive_df_clean.pupper + \
    twitter_archive_df_clean.puppo

In [ ]:
# check on the new column
twitter_archive_df_clean.stage.value_counts()

In [ ]:
twitter_archive_df_clean.stage.value_counts()

In [15]:

# deal with the rows with multiple stage values
twitter_archive_df_clean.loc[twitter_archive_df_clean.stage == 'doggopupper'] = 'doggo,pupper'
twitter_archive_df_clean.loc[twitter_archive_df_clean.stage == 'doggofloofer'] = 'doggo,fluffer'
twitter_archive_df_clean.loc[twitter_archive_df_clean.stage == 'doggopuppo'] = 'doggo,puppo'


In [ ]:
twitter_archive_df_clean.stage.value_counts()

In [ ]:
twitter_archive_df_clean.stage.value_counts()

In [16]:
# convert blank name cells to np.nan...but it seems like this turns everything into nan

twitter_archive_df_clean.stage.replace('', np.nan, inplace=True)

#old code...twitter_archive_df_clean.loc[twitter_archive_df_clean.stage == ''] = np.nan

In [17]:
# drop old columns
twitter_archive_df_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

In [20]:
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   tweet_id                    2356 non-null   object
 1   in_reply_to_status_id       91 non-null     object
 2   in_reply_to_user_id         91 non-null     object
 3   timestamp                   2356 non-null   object
 4   source                      2356 non-null   object
 5   text                        2356 non-null   object
 6   retweeted_status_id         193 non-null    object
 7   retweeted_status_user_id    193 non-null    object
 8   retweeted_status_timestamp  193 non-null    object
 9   expanded_urls               2297 non-null   object
 10  rating_numerator            2356 non-null   object
 11  rating_denominator          2356 non-null   object
 12  name                        1512 non-null   object
 13  stage                       380 non-null    obje

In [ ]:
# 7. merge all tables into one master table

In [18]:
# first change tweet id columns to object datatype (in the other tables)
final_json_data = final_json_data.astype({'id': 'object'})
image_predictions_df = image_predictions_df.astype({'tweet_id': 'object'})


In [19]:
# for consistency, change 'id' in json_data_df to 'tweet_id'
final_json_data.rename(columns = {'id':'tweet_id'}, inplace = True)

In [20]:
# merge final_json_data and image_predictions_df
result_1 = pd.merge(image_predictions_df,final_json_data,on='tweet_id')
result_1.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True,532,2535
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True,48,132
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,47,128
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,147,311
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True,41,111


In [22]:
# merge twitter_archive_df_clean and 'result_1' into master_df
master_df = pd.merge(twitter_archive_df_clean,result_1,on='tweet_id')


In [23]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2060 entries, 0 to 2059
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2060 non-null   object 
 1   in_reply_to_status_id       22 non-null     object 
 2   in_reply_to_user_id         22 non-null     object 
 3   timestamp                   2060 non-null   object 
 4   source                      2060 non-null   object 
 5   text                        2060 non-null   object 
 6   retweeted_status_id         77 non-null     object 
 7   retweeted_status_user_id    77 non-null     object 
 8   retweeted_status_timestamp  77 non-null     object 
 9   expanded_urls               2060 non-null   object 
 10  rating_numerator            2060 non-null   object 
 11  rating_denominator          2060 non-null   object 
 12  name                        1392 non-null   object 
 13  stage                       307 n

In [24]:
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   tweet_id                    2356 non-null   object
 1   in_reply_to_status_id       91 non-null     object
 2   in_reply_to_user_id         91 non-null     object
 3   timestamp                   2356 non-null   object
 4   source                      2356 non-null   object
 5   text                        2356 non-null   object
 6   retweeted_status_id         193 non-null    object
 7   retweeted_status_user_id    193 non-null    object
 8   retweeted_status_timestamp  193 non-null    object
 9   expanded_urls               2297 non-null   object
 10  rating_numerator            2356 non-null   object
 11  rating_denominator          2356 non-null   object
 12  name                        1512 non-null   object
 13  stage                       380 non-null    obje

In [24]:
master_df.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
874,739238157791694849,NaN,NaN,2016-06-04 23:31:25+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo blowing bubbles. It's downright...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/739238157...,...,0.503372,True,Siberian_husky,0.390413,True,malamute,0.080901,True,52360,75163
55,880221127280381952,NaN,NaN,2017-06-29 00:27:25+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jesse. He's a Fetty Woof. His tongue ejec...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/880221127...,...,0.238525,True,meerkat,0.104256,False,clumber,0.052580,True,4436,27640
581,781661882474196992,NaN,NaN,2016-09-30 01:08:10+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Who keeps sending in pictures without dogs in ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/781661882...,...,0.438087,True,golden_retriever,0.226954,True,collie,0.070652,True,3129,11634


In [25]:
master_df.tweet_id.duplicated().sum()

0

In [24]:
# 2. change timestamp column to datetime datatype
master_df["timestamp"]= pd.to_datetime(master_df["timestamp"])

In [25]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2060 entries, 0 to 2059
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2060 non-null   object             
 1   in_reply_to_status_id       22 non-null     object             
 2   in_reply_to_user_id         22 non-null     object             
 3   timestamp                   2060 non-null   datetime64[ns, UTC]
 4   source                      2060 non-null   object             
 5   text                        2060 non-null   object             
 6   retweeted_status_id         77 non-null     object             
 7   retweeted_status_user_id    77 non-null     object             
 8   retweeted_status_timestamp  77 non-null     object             
 9   expanded_urls               2060 non-null   object             
 10  rating_numerator            2060 non-null   object          

#### Test

In [ ]:
# 0. 
twitter_archive_df_clean.head(3)

In [17]:
# 1. change id columns to object datatype
print(twitter_archive_df_clean.dtypes)

tweet_id                      object
in_reply_to_status_id         object
in_reply_to_user_id           object
timestamp                     object
source                        object
text                          object
retweeted_status_id           object
retweeted_status_user_id      object
retweeted_status_timestamp    object
expanded_urls                 object
rating_numerator               int64
rating_denominator             int64
name                          object
doggo                         object
floofer                       object
pupper                        object
puppo                         object
dtype: object


In [14]:
2.
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       78 non-null     object             
 2   in_reply_to_user_id         78 non-null     object             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    object             
 7   retweeted_status_user_id    181 non-null    object             
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

In [8]:
# 3. for the row where denominator has a rating of "0", change it "10"
twitter_archive_df_clean.loc[[313]]

NameError: name 'twitter_archive_df_clean' is not defined

In [13]:
# 4. convert rows (where the dog name is 'None') to np.nan
twitter_archive_df_clean.loc[twitter_archive_df_clean['name']=="None"]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [30]:
# 6
twitter_archive_df_clean[['name','stage']].sample(5)

,name,stage
1369,Rudy,NaN
1145,Karll,NaN
1262,Tater,NaN
1995,Scott,pupper
1268,Cecil,NaN


In [54]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2060 entries, 0 to 2059
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2060 non-null   object 
 1   in_reply_to_status_id       22 non-null     object 
 2   in_reply_to_user_id         22 non-null     object 
 3   timestamp                   2060 non-null   object 
 4   source                      2060 non-null   object 
 5   text                        2060 non-null   object 
 6   retweeted_status_id         77 non-null     object 
 7   retweeted_status_user_id    77 non-null     object 
 8   retweeted_status_timestamp  77 non-null     object 
 9   expanded_urls               2060 non-null   object 
 10  rating_numerator            2060 non-null   object 
 11  rating_denominator          2060 non-null   object 
 12  name                        1392 non-null   object 
 13  stage                       307 n

## Export the master dataframe

In [22]:
# saving the DataFrame as a CSV file
test_file = master_df.to_csv('twitter_archive_master.csv', index = False)

In [28]:
# saving the DataFrame to SQLite db
import sqlite3

conn = sqlite3.connect('twitter_archive_master_copy.db') 
master_df.to_sql('master_table', conn, if_exists='replace')

## Analysis/Insights

In [26]:
master_df = master_df.astype({"rating_numerator": float,"rating_denominator": float})

In [27]:
master_df['final_score'] = master_df['rating_numerator'] / master_df['rating_denominator'] 

In [28]:
test = master_df.loc[master_df['name']=='Atticus']

In [29]:
test[['timestamp', 'name',  'rating_numerator', 'rating_denominator', 'stage', 'tweet_id', ]]

,timestamp,name,rating_numerator,rating_denominator,stage,tweet_id
791,2016-07-04 15:00:45+00:00,Atticus,1776.0,10.0,NaN,749981277374128128
818,2016-06-27 00:07:44+00:00,Atticus,11.0,10.0,NaN,747219827526344708


In [41]:
master_df['final_score'].nlargest(5)

791     177.600000
1782     42.000000
552       7.500000
412       3.428571
607       2.700000
Name: final_score, dtype: float64

In [48]:
master_df.iloc[['791', '1782','552','412','607']].name

791     Atticus
1782        NaN
552       Logan
412         Sam
607      Sophie
Name: name, dtype: object